In [1]:
from HSI_utils import *
import conv_models
import conv_models_AD
import conv_models_AD_full_HSI
import dense_models
from losses import *
from timeit import default_timer as timer

from datetime import datetime, timedelta
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

In [2]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
if torch.version.cuda != None:
    # Storing ID of current CUDA device
    cuda_id = torch.cuda.current_device()
    print(f"ID of current CUDA device: {torch.cuda.current_device()}")
    print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 11.7
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce GTX 1070


In [3]:
# random_img = torch.from_numpy(np.float32(generate_random_HSI(1,1,207)))
random_img = torch.from_numpy(np.float32(generate_random_HSI(20, 20, 8)))
random_img = torch.permute(random_img, (2, 0, 1))
random_img = torch.unsqueeze(random_img, dim=0)
random_img = torch.squeeze(random_img, -1)
# random_img = torch.squeeze(random_img, -1)
img_shape = torch.Tensor.size(random_img)
print(img_shape)

torch.Size([1, 8, 20, 20])


In [4]:
zdim = 128
n_channels = 8
spatial_dim = 20

##ENCODER

print(f'Input original HSI image shape: {np.shape(random_img)}')

# encoder = conv_models_AD_full_HSI.Encoder(n_channels=n_channels, zdim=zdim, spatial_dim=spatial_dim, verbose=1)

encoder = conv_models_AD.Encoder(n_channels=n_channels, zdim=zdim, spatial_dim=spatial_dim, verbose=1)

# encoder = dense_models.Encoder(zdim=32, 
#                                dense_out_1=256, 
#                                n_channels=207, 
#                                n_layers=2, 
#                                variational=True,
#                                activation = 'gelu', 
#                                reduction_factor = 2, 
#                                verbose = 1)

# encoder = conv_models.Encoder(n_channels=img_shape[2], 
                #   fout1 = 256, 
                #   zdim = 256, 
                #   variational=True, 
                #   n_conv_blocks=2, 
                #   spatial_dim=5, 
                #   GELU = True,
                #   conv_factor = 2,
                #   verbose = 0)

if torch.cuda.is_available():
    random_img = random_img.to('cuda')
    encoder = encoder.to('cuda')

print(encoder)

z, z_mu, z_logvar, dense_outs = encoder(random_img)

KL_div = kl_loss(z_mu, z_logvar)
print(f'KL divergence = {KL_div}')

## DECODER

# random_latent_vector = torch.from_numpy(np.float32(np.random.rand(256)))
# z_shape = torch.Tensor.size(random_latent_vector)

# decoder = conv_models_AD_full_HSI.Decoder(zdim=zdim, n_channels=n_channels, spatial_dim=spatial_dim, verbose=1)

decoder = conv_models_AD.Decoder(zdim=zdim, n_channels=n_channels, spatial_dim=spatial_dim, verbose=1)

# decoder = dense_models.Decoder(zdim=32, 
#                                dense_out_1=256, 
#                                n_channels=207, 
#                                n_layers=2, 
#                                activation = 'gelu', 
#                                reduction_factor = 2, 
#                                verbose = 1) 

# decoder = conv_models.Decoder(zdim=256, 
#                                fout1=256, 
#                                n_channels=207,  
#                                n_conv_blocks=2, 
#                                spatial_dim=5, 
#                                GELU = True, 
#                                conv_factor = 2, 
#                                verbose = 1)

if torch.cuda.is_available():
    # random_latent_vector = random_latent_vector.to('cuda')
    decoder = decoder.to('cuda')

print(decoder)


# out, act_maps = convolutional_decoder(random_latent_vector)
img_hat, dec_layer_outs = decoder(z)


print(f'Output reconstructed HSI image shape: {np.shape(img_hat)}')

Input original HSI image shape: torch.Size([1, 8, 20, 20])
Encoder(
  (conv0): Conv2d(8, 128, kernel_size=(1, 1), stride=(1, 1))
  (norm0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
  (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (norm3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv35): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm35): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (norm4): BatchNorm2d(1024, 